In [1]:
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.io import ascii
import pandas as pd
import os
from astropy.table import Table, Column
import numpy as np
import matplotlib.pyplot as plt
import time
#import seaborn as sns
%matplotlib inline

The USNO-B catalog is "I/284" 

In [2]:
data=Table.read("/Users/kschles/Documents/GALAH/HVS/update_08112015/sobject_iraf_general.fits")

In [3]:
data=data.to_pandas()

In [4]:
#Output smaller file to use to query USNO via STILTS and TAP
data[['sobject_id', 'ra', 'dec']].to_csv('propermotion_temp.csv', index=False)

In [5]:
### Want to find all matches in USNO-B within 7" of the target
os.system('java -jar stilts.jar cdsskymatch cdstable=I/284/out find=all in=propermotion_temp.csv ifmt=csv ra=ra dec=dec radius=7 out=usnob_7as.csv')

0

In [6]:
## Read in the 7" radius query and use it to identify nearest neighbors and matches. 
usnob=pd.read_csv('/Users/kschles/Documents/GALAH/propermotion/usnob_7as.csv')

In [7]:
usnob.loc[np.where(usnob['angDist']<=1.0)[0], 'match']=1
usnob.loc[np.where(((usnob['Bmag1']<22.) | (usnob['Bmag2']<22.)) & (usnob['angDist']>1.0))[0], 'neighbor']=1

In [8]:
test=usnob.groupby(['sobject_id'])

In [9]:
prelim_quality=test.agg({'sobject_id': 'first', 'match' : np.sum, 'neighbor' : np.sum, 'angDist' : lambda x: min(x)})

In [10]:
combo1=pd.merge(data, prelim_quality[['sobject_id', 'match', 'neighbor']], on=['sobject_id'], how='left')

In [13]:
# Need to make a loop to query the usno-b catalog using tap because has issues with searches greater than 1750 items. 

usno_index=np.append([np.arange(0,len(data),1750)],[len(data)-1])
#for j in range(1,len(usno_index)) :
for j in range(1,10) :
    print usno_index[j-1], time.ctime()
    if (j!=1):
        data.loc[usno_index[j-1]+1:usno_index[j],['sobject_id', 'ra', 'dec']].to_csv('propermotion_temp.csv', index=False)
    if (j==1): 
        data.loc[usno_index[j-1]:usno_index[j],['sobject_id', 'ra', 'dec']].to_csv('propermotion_temp.csv', index=False)
    temp=os.system('java -jar stilts.jar tapskymatch tapurl=http://dc.g-vo.org/tap taptable=usnob.data in=propermotion_temp.csv ifmt=csv inlon=ra inlat=dec out=usnob_2as_temp.csv taplon=RAJ2000 taplat=DEJ2000 sr=0.00050 find=each') 
    os.system('mv usnob_2as_temp.csv usnob_output/usnob_2as_'+str(j).zfill(3)+'.csv')


0 Fri May  6 16:54:12 2016
1750 Fri May  6 16:54:22 2016
3500 Fri May  6 16:54:30 2016
5250 Fri May  6 16:54:37 2016
7000 Fri May  6 16:54:45 2016
8750 Fri May  6 16:54:51 2016
10500 Fri May  6 16:54:59 2016
12250 Fri May  6 16:55:06 2016
14000 Fri May  6 16:55:14 2016


In [14]:
# Read in and combine the USNO-B files created using TAP
#for j in range(1,len(usno_index)) :
for j in range(1,10) :
    filename='usnob_output/usnob_2as_'+str(j).zfill(3)+'.csv'
    temp=pd.read_csv(filename)
    if j==1: 
        usnob_best=temp
    if j>1: 
        usnob_best=pd.concat([usnob_best, temp])

In [15]:
## Merge teh USNO-B data with the rest
combo=pd.merge(combo1,usnob_best[['sobject_id', 'fit_ra', 'fit_de', 'ndet', 'sep_arcsec']],\
                            on=['sobject_id'], how='left')

In [16]:
# There should be no rows here! 
combo.loc[np.where((combo['match']>=1) & (np.isnan(combo['ndet'])==True))[0],\
                   ['sobject_id', 'RADEG', 'DEDEG']]

,sobject_id,RADEG,DEDEG
15751,1.403030e+14,NaN,NaN
15752,1.403030e+14,NaN,NaN
15753,1.403030e+14,NaN,NaN
15754,1.403030e+14,NaN,NaN
15755,1.403030e+14,NaN,NaN
15757,1.403030e+14,NaN,NaN
15758,1.403030e+14,NaN,NaN
15759,1.403030e+14,NaN,NaN
15760,1.403030e+14,NaN,NaN
15761,1.403030e+14,NaN,NaN


In [18]:
#Adjust the fit_RA and fit_DE values from degrees to mas
combo['fit_ra']=np.round(combo['fit_ra']*3600.*1000.)
combo['fit_de']=np.round(combo['fit_de']*3600.*1000.)


In [19]:
combo_table=Table.from_pandas(combo)

In [29]:
combo_table['ra'].unit='deg'
combo_table['dec'].unit='deg'
combo_table['eff_midtexp'].unit='s'
combo_table['eff_texp'].unit='s'
combo_table['barycentric'].unit='km/s'
combo_table['ebv'].unit='mag'
combo_table['fit_ra'].unit='mas'
combo_table['fit_de'].unit='mas'
combo_table['sep_arcsec'].unit='as'

combo_table['j_tmass'].unit='mag'
combo_table['e_j_tmass'].unit='mag'
combo_table['h_tmass'].unit='mag'
combo_table['e_h_tmass'].unit='mag'
combo_table['k_tmass'].unit='mag'
combo_table['e_k_tmass'].unit='mag'

combo_table['b_apass'].unit='mag'
combo_table['e_b_apass'].unit='mag'
combo_table['v_apass'].unit='mag'
combo_table['e_v_apass'].unit='mag'
combo_table['g_apass'].unit='mag'
combo_table['e_g_apass'].unit='mag'
combo_table['r_apass'].unit='mag'
combo_table['e_r_apass'].unit='mag'
combo_table['i_apass'].unit='mag'
combo_table['e_i_apass'].unit='mag'

combo_table['pmra_ucac4'].unit='mas/yr'
combo_table['e_pmra_ucac4'].unit='mas/yr'
combo_table['pmdec_ucac4'].unit='mas/yr'
combo_table['e_pmdec_ucac4'].unit='mas/yr'

In [30]:
combo_table.write('sobject_iraf_pmquality.fits')

In [55]:
combo.to_csv('propermotion_quality_05062016.csv', index=False)